# Import modules

In [1]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import math
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier, XGBRegressor
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
import catboost as cb
import optuna
from sklearn.model_selection import StratifiedKFold, cross_val_score
import pprint
from copy import deepcopy
from typing import *
import lightgbm as lgb
from catboost import CatBoostClassifier
from functools import reduce
from sklearn.preprocessing import TargetEncoder
from ktools.preprocessing.categorical_denoiser_prepreprocesser import CategoricalDenoiserPreprocessor
from ktools.metrics.fast_matthew_correlation_coefficient import fast_matthews_corr_coeff
from ktools.preprocessing.categorical_string_label_error_imputator import CategoricalLabelErrorImputator
from ktools.preprocessing.categorical_features_embedder import SortMainCategories
from ktools.preprocessing.kaggle_dataset_manager import KaggleDatasetManager
from ktools.utils.data_science_pipeline_settings import DataSciencePipelineSettings
from sklearn.linear_model import LogisticRegression
from ktools.fitting.cross_validate_then_test_sklearn_model import CrossValidateTestSklearnModel
from sklearn.metrics import root_mean_squared_error, r2_score
from sklearn.model_selection import KFold

/Users/yuwei-1/anaconda3/envs/ktools/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Main

In [2]:
class ConvertToLower:
    @staticmethod
    def transform(settings : DataSciencePipelineSettings):
        for col_name in settings.categorical_col_names:
            settings.combined_df[col_name] = settings.combined_df[col_name].str.lower()
        return settings

In [22]:
class UsedCarSpecificConverter:
    @staticmethod
    def transform(original_settings : DataSciencePipelineSettings):
        settings = deepcopy(original_settings)
        def find_pattern(pattern, text):
            match = re.search(pattern, text)
            if match:
                return match.group(1)
            else:
                return None
            
        def transmission(text):
            if 'a/t' in text or 'at' in text or 'automatic' in text:
                return 'automatic transmission'
            elif 'm/t' in text or 'mt' in text or 'manual' in text:
                return 'manual transmission'
            elif 'cvt' in text:
                return 'continuously variable transmission'
            else:
                return 'other'

        def camshafts(text):
            if 'dohc' in text:
                # double overhead camshaft
                return 'dohc'
            elif 'sohc' in text:
                #single overhead camshaft
                return 'sohc'
            elif 'ohv' in text:
                # overhead valve
                return 'ohv'
            elif 'vtec' in text:
                # variable valve timing and lift electronic control
                return 'vtec'
            else:
                return 'other'

        def injection(text):
            if 'ddi' in text:
                #direct diesel injection
                return 'ddi'
            elif 'gdi' in text:
                #gasoline direct injection
                return 'gdi'
            elif 'mpfi' in text:
                # multi-point fuel injection
                return 'mpfi'
            elif 'pdi' in text:
                # port fuel injection
                return 'pdi'
            elif 'tfsi' in text or 'tsi' in text:
                # turbo stratified injection
                return 'tfsi'
            elif 'gtdi' in text:
                # gasoline turbocharged direct injection
                return 'gtdi'
            elif 'sidi' in text:
                # spark ignition direct injection
                return 'sidi'
            else:
                return 'other'
            
        pattern = r'(\d*\.?\d+)\s*hp'
        settings.combined_df['horsepower'] = settings.combined_df['engine'].apply(lambda x : find_pattern(pattern, x)).astype('float64')
        pattern = r'(\d*\.?\d+)\s*(l|liter)'
        settings.combined_df['liters'] = settings.combined_df['engine'].apply(lambda x : find_pattern(pattern, x)).astype('float64')
        pattern = r'(\d*\.?\d+)\s*cylinder'
        settings.combined_df['cylinders'] = settings.combined_df['engine'].apply(lambda x : find_pattern(pattern, x)).astype('float64')
        pattern = r'(\d*\.?\d+)\s*(-speed|speed)'
        settings.combined_df['speed'] = settings.combined_df['transmission'].apply(lambda x : find_pattern(pattern, x)).astype('float64')
        
        settings.combined_df['injection'] = settings.combined_df['engine'].apply(lambda x : injection(x)).astype('category')
        settings.combined_df['camshaft'] = settings.combined_df['engine'].apply(lambda x : camshafts(x)).astype('category')
        settings.combined_df['transmission_clean'] = settings.combined_df['transmission'].apply(lambda x : transmission(x)).astype('category')
        settings.combined_df.loc[(settings.combined_df['model'].str.contains('model y|model x|model s|model 3', regex=True)), 'fuel_type'] = 'electric'
        settings.combined_df.loc[(settings.combined_df['model'].str.contains('electric')), 'fuel_type'] = 'electric'


        expensive_ext_color = ['blue caelum','dark sapphire','bianco monocerus','c / c',
                               'ice','tempest','beluga black','bianco icarus metallic','blu eleos',
                               'shadow black','nero noctis','sandstone metallic','lizard green','balloon white','onyx',
                               'donington grey metallic','china blue','diamond white','rosso corsa',
                                'granite','rosso mars metallic',
                                'carpathian grey','kemora gray metallic','grigio nimbus','dash','bianco isis','python green',
                                'fountain blue','custom color','vega blue','designo magno matte',
                                'brands hatch gray metallic',
                                'rift metallic','gentian blue metallic',
                                'arancio borealis','blue',
                                'aventurine green metallic',
                                'apex blue','daytona gray pearl effect',
                                'daytona gray pearl effect w/ black roof','matte white',
                                'carpathian grey premium metallic','blue metallic','santorini black metallic',
                                'quartzite grey metallic','carrara white metallic','black',
                                'kinetic blue',
                                'nero daytona']

        expensive_int_color = ['dark auburn',
                            'hotspur',
                            'cobalt blue',
                            'beluga hide',
                            'linen',
                            'beluga',
                            'black / brown',
                            'nero ade',
                            'sahara tan',
                            'portland']
        
        
        
        settings.combined_df['expensive_ext_col'] = settings.combined_df['ext_col'].isin(expensive_ext_color).astype(int)
        settings.combined_df['expensive_int_col'] = settings.combined_df['int_col'].isin(expensive_int_color).astype(int)
        settings.combined_df['twin_turbo'] = settings.combined_df['engine'].str.contains('twin turbo').astype(int)
        settings.combined_df['turbo'] = settings.combined_df['engine'].str.contains('turbo').astype(int)
        
        clean_colors = ['ext_col', 'int_col']
        string_imputator = CategoricalLabelErrorImputator(verbose=True)
        settings.combined_df[['basic_ext_color', 'basic_int_color']] = string_imputator.impute(settings.combined_df[clean_colors],
                                                                                                                    clean_colors,
                                                                                                                    1500)
        
        settings.combined_df['basic_ext_color'] = settings.combined_df['basic_ext_color'].astype('category')
        settings.combined_df['basic_int_color'] = settings.combined_df['basic_int_color'].astype('category')
        # threshold = 2.5e6
        # print("num removed: ", (settings.combined_df['price'] > threshold).sum())
        # settings.combined_df = settings.combined_df[(settings.combined_df['price'] < threshold) | (settings.combined_df['price'].isna())]
        
        settings.training_col_names += ['horsepower', 
                                        'injection', 
                                        'camshaft', 
                                        'cylinders', 
                                        'expensive_ext_col', 
                                        'expensive_int_col', 
                                        'twin_turbo', 
                                        'turbo',
                                        'transmission_clean',
                                        'speed',
                                        'basic_ext_color',
                                        'basic_int_color'
                                        ]
        
        settings.categorical_col_names += ['injection',
                                           'camshaft',
                                           'transmission_clean',
                                           'basic_ext_color',
                                           'basic_int_color']
        
        return settings

In [84]:
class SergeyConverter:
    @staticmethod
    def transform(original_settings : DataSciencePipelineSettings):
        settings = deepcopy(original_settings)
        fuel_type_dict = {
            'Gasoline': 0,
            'Hybrid': 1,
            'E85 Flex Fuel': 2,
            'uknown': 3,
            'Diesel': 4,
            'dash': 5,
            'Plug-In Hybrid': 6,
            'not supported': 7
        }

        accident_dict = {
            'None reported': 0,
            'At least 1 accident or damage reported': 1,
            'uknown': 2
        }

        clean_title_dict = {
            'Yes': 0,
            'uknown': 1
        }

        expensive_ext_color = ['Blue Caelum', 'Dark Sapphire', 'Bianco Monocerus', 'C / C', 'Ice',
            'Tempest', 'Beluga Black', 'Bianco Icarus Metallic', 
            'BLU ELEOS', 'Shadow Black', 'Nero Noctis', 'Sandstone Metallic',
            'Lizard Green', 'Balloon White', 'Onyx', 'Donington Grey Metallic',
            'China Blue', 'Diamond White', 'Rosso Corsa', 'Granite',
            'Rosso Mars Metallic', 'Carpathian Grey', 'Kemora Gray Metallic',
            'Grigio Nimbus', 'dash', 'Bianco Isis', 'Python Green', 'Fountain Blue',
            'Custom Color', 'Vega Blue', 'Designo Magno Matte',
            'Brands Hatch Gray Metallic', 'Rift Metallic', 'Gentian Blue Metallic',
            'Arancio Borealis', 'BLUE', 'Aventurine Green Metallic', 'Apex Blue',
            'Daytona Gray Pearl Effect', 'Daytona Gray Pearl Effect w/ Black Roof',
            'Matte White', 'Carpathian Grey Premium Metallic', 'Blue Metallic',
            'Santorini Black Metallic', 'Quartzite Grey Metallic',
            'Carrara White Metallic', 'BLACK', 'Kinetic Blue', 'Nero Daytona']

        expensive_int_color = ['Dark Auburn', 'Hotspur', 'Cobalt Blue', 'Beluga Hide', 'Linen',
                            'Beluga', 'Black / Brown', 'Nero Ade', 'Sahara Tan', 'Portland']

        expensive_hp = [443.0, 473.0, 493.0, 502.0, 521.0, 542.0, 543.0, 571.0, 572.0, 573.0, 580.0,
                        591.0, 602.0, 611.0, 616.0, 620.0, 624.0, 640.0, 641.0, 651.0, 710.0, 715.0, 760.0, 788.0, 797.0]



        def encode_columns(df):
            df['fuel_type_encoded'] = df['fuel_type'].map(fuel_type_dict)
            df['accident_encoded'] = df['accident'].map(accident_dict)
            df['clean_title_encoded'] = df['clean_title'].map(clean_title_dict)
            df['expensive_color_ext_encoded'] = df.ext_col.isin(expensive_ext_color).astype(int)
            df['expensive_color_int_encoded'] = df.int_col.isin(expensive_int_color).astype(int)
            df['expensive_hp'] = df['engine'].str.extract(r'(\d+\.?\d*)HP').astype(float).isin(expensive_hp).astype(int)
            df['cylinder'] = df['engine'].str.extract(r'(\d+\.?\d*) Cylinder').astype(float)              
            df['got_V'] = df['model'].str.extract(r'(\d+\.?\d*) V').notna().astype(int)
            return df
        settings.combined_df = encode_columns(settings.combined_df)
        settings.training_col_names += ['expensive_color_ext_encoded',
                                        'expensive_color_int_encoded']
        return settings

In [4]:
class FillNullValues:
    @staticmethod
    def transform(settings : DataSciencePipelineSettings, numeric_fill=-1, category_fill='missing'):
        for col_name in settings.training_col_names:
            if pd.api.types.is_numeric_dtype(settings.combined_df[col_name]):
                settings.combined_df[col_name] = settings.combined_df[col_name].fillna(numeric_fill)
            else:
                settings.combined_df[col_name] = settings.combined_df[col_name].fillna(category_fill)
        return settings

In [5]:
class ConvertObjectToCategorical:
    @staticmethod
    def transform(settings : DataSciencePipelineSettings):
        cat_cols = settings.categorical_col_names
        settings.combined_df[cat_cols] = settings.combined_df[cat_cols].astype('category')
        return settings

In [6]:
class LogTransformTarget:
    @staticmethod
    def transform(original_settings : DataSciencePipelineSettings):
        settings = deepcopy(original_settings)
        target = settings.target_col_name
        settings.combined_df['log_' + target] = np.log(settings.combined_df[target] + 1)
        settings.target_col_name = 'log_' + target
        settings.logged = True
        return settings

In [23]:
train_csv_path = "data/used_car_prices/train.csv"
test_csv_path = "data/used_car_prices/test.csv"
target_col_name = "price"

settings = DataSciencePipelineSettings(train_csv_path,
                                        test_csv_path,
                                        target_col_name)

full_transforms = [ConvertToLower.transform, FillNullValues.transform, UsedCarSpecificConverter.transform, ConvertObjectToCategorical.transform, LogTransformTarget.transform]
# basic_transforms = [ConvertToLower.transform, FillNullValues.transform, ConvertObjectToCategorical.transform, LogTransformTarget.transform]

full_settings = reduce(lambda acc, func: func(acc), full_transforms, settings)
# basic_settings = reduce(lambda acc, func: func(acc), basic_transforms, settings)

(original : new category) {'bright white clearcoat': 'white', 'diamond black': 'black', 'agate black metallic': 'black', 'summit white': 'white', 'silver ice metallic': 'silver', 'mythos black metallic': 'black', 'midnight black metallic': 'black', 'oxford white': 'white', 'santorini black metallic': 'black', 'black clearcoat': 'black', 'alpine white': 'white', 'obsidian black metallic': 'black', 'designo diamond white metallic': 'white', 'snowflake white pearl': 'white', 'firecracker red clearcoat': 'red', 'soul red crystal metallic': 'red', 'daytona gray pearl effect': 'gray', 'delmonico red pearlcoat': 'red', 'glacial white pearl': 'white', 'shadow gray metallic': 'gray', 'atomic silver': 'silver', 'fuji white': 'white', 'black sapphire metallic': 'black', 'velvet red pearlcoat': 'red', 'santorin black': 'black', 'cajun red tintcoat': 'red', 'lunar silver metallic': 'silver', 'black raven': 'black', 'brilliant silver metallic': 'silver', 'ebony black': 'black', 'daytona gray pearl e

In [130]:
# train_df, test_df = basic_settings.update()

# data_manager = KaggleDatasetManager(train_df,
#                                     basic_settings.training_col_names,
#                                     basic_settings.target_col_name,
#                                     0.9,
#                                     0.1,
#                                     0)

# (X_train, 
# X_valid, 
# X_test, 
# y_train, 
# y_valid,
# y_test) = data_manager.dataset_partition()


# model = XGBRegressor(**{}, 
#                     verbosity=0,
#                     eval_metric='logloss',
#                     tree_method='hist',
#                     enable_categorical=True)
# num_splits = 10
# eval_metrics = {"r2" : r2_score, "rmse" : lambda y, yhat : root_mean_squared_error(np.exp(y), np.exp(yhat))}
# skf = KFold(n_splits=num_splits)

# model, cv_scores, test_scores = CrossValidateTestSklearnModel(model,
#                               eval_metrics,
#                               skf,
#                               num_splits).evaluate(X_train,
#                                                    y_train,
#                                                    X_test,
#                                                    y_test)

Final Model r2: 0.638607

10-fold cross validation r2:  0.6376903986890982

Final Model rmse: 70167.345097

10-fold cross validation rmse:  73703.32201325778

In [24]:
train_df, test_df = full_settings.update()

data_manager = KaggleDatasetManager(train_df,
                                    full_settings.training_col_names,
                                    full_settings.target_col_name,
                                    0.9,
                                    0.1,
                                    0)

(X_train, 
X_valid, 
X_test, 
y_train, 
y_valid,
y_test) = data_manager.dataset_partition()


model = XGBRegressor(**{}, 
                    verbosity=0,
                    eval_metric='logloss',
                    tree_method='hist',
                    enable_categorical=True)
num_splits = 10
eval_metrics = {"r2" : r2_score, "rmse" : lambda y, yhat : root_mean_squared_error(np.exp(y) - 1, np.exp(yhat) - 1)}
skf = KFold(n_splits=num_splits)

model, cv_scores, test_scores = CrossValidateTestSklearnModel(model,
                              eval_metrics,
                              skf,
                              num_splits).evaluate(X_train,
                                                   y_train,
                                                   X_test,
                                                   y_test)

Final Model r2: 0.640399
10-fold cross validation r2:  0.6396663903397439
Final Model rmse: 70122.085231
10-fold cross validation rmse:  73616.34759331397


In [25]:
train_df, test_df = full_settings.update()
train_df.to_csv('/Users/yuwei-1/Documents/projects/Kaggle-tools/data/used_car_prices/train_v1.csv')
test_df.to_csv('/Users/yuwei-1/Documents/projects/Kaggle-tools/data/used_car_prices/test_v1.csv')

In [253]:
sub = pd.read_csv('/Users/yuwei-1/Documents/projects/Kaggle-tools/data/used_car_prices/sample_submission.csv', index_col=0)
sub['price'] = np.exp(model.predict(test_df[full_settings.training_col_names])) - 1
sub

,price
id,
188533,14647.727539
188534,75897.273438
188535,48129.507812
188536,19080.269531
188537,28636.031250
...,...
314218,22809.539062
314219,47389.597656
314220,17833.939453


In [262]:
pd.read_csv('/Users/yuwei-1/Documents/projects/Kaggle-tools/submissions/used_cars/trupolog_ag.csv')['price'].max()

307631.2

In [254]:
sub.to_csv('submissions/used_cars/used_car_submission_v5.csv')

In [182]:
import itertools


set(itertools.chain.from_iterable([x.split() for x in full_settings.combined_df['ext_col'].tolist()]))

{'1-speed',
 '10-speed',
 '2',
 '2-speed',
 '4-speed',
 '5-speed',
 '6',
 '6-speed',
 '7-speed',
 '8-speed',
 '9-speed',
 'a/t',
 'at',
 'at/mt',
 'auto-shift',
 'automatic',
 'controlled',
 'cvt',
 'cvt-f',
 'dct',
 'electronically',
 'f',
 'fixed',
 'for',
 'gear',
 'in',
 'm/t',
 'manual',
 'mode',
 'mt',
 'o',
 'or',
 'overdrive',
 'production',
 'scheduled',
 'shift',
 'single-speed',
 'speed',
 'switch',
 'transmission',
 'variable',
 'w/dual',
 'with',
 '–'}